In [27]:
# give 2 portforlio, the old one and the optimized new one, 
# calculate the stepwise change of the portfolio
# and prioritize the changes by the improvement of the portfolio

cur_investment = [

        {
            "stock": "GILD",
            "quantity": 55,
        },
        {
            "stock": "LLY",
            "quantity": 6.68,
        },
        {
            "stock": "NVR",
            "quantity": 0.52,
        },
        {
            "stock": "MCK",
            "quantity": 6.56,
        },
        {
            "stock": "STAN.L",
            "quantity": 196.2
        },
        {
            "stock": "JKHY",
            "quantity": 30
        },
        {
            "stock": "601225.SS",
            "quantity": 600
        },
        {
            "stock": "600276.SS",
            "quantity": 300
        },
        {
            "stock": "600900.SS",
            "quantity": 600
        },
        {
            "stock": "MRK",
            "quantity": 11.67
        },
        {
            "stock": "VRTX",
            "quantity": 3.07
        },
        {
            "stock": "WMT",
            "quantity": 37.4
        },
        {
            "stock": "CTRA",
            "quantity": 64.99
        }
    ]


In [28]:
# compute the portfolio value

from util import get_currency_pair, read_and_filter_exchange_rates

tot_value = 0
for stock in cur_investment:
    # get the price of the stock which is the last line in the stock price file
    file_path = './data/prices/' + stock["stock"] + '.csv'
    with open(file_path) as f:
        lines = f.readlines()
        stock_price = float(lines[-1].split(",")[1])
        # if the price is zero, use the previous price
        if stock_price == 0:
            stock_price = float(lines[-2].split(",")[1])

    if '.' in stock["stock"]:
        stock_suffix = '.' + stock["stock"].split(".")[-1]
        # convert GBX to GBP
        if stock_suffix == '.L':
            stock_price *= 0.01
        exchange_name, needs_inversion, exchange_name_yahoo = get_currency_pair(stock_suffix, "USD")
        df_rate = read_and_filter_exchange_rates(exchange_name, exchange_name_yahoo)
        #print(df_rate)
        rate = df_rate[exchange_name].iloc[-1]

        if needs_inversion:
            rate = 1 / rate

        print(stock["stock"], stock_price, stock_price * rate, stock["quantity"] * stock_price * rate)
        tot_value += stock["quantity"] * stock_price * rate
        stock['value'] = stock["quantity"] * stock_price * rate
    else:
        print(stock["stock"], stock_price, stock["quantity"] * stock_price)
        tot_value += stock["quantity"] * stock_price
        stock['value'] = stock["quantity"] * stock_price


print(tot_value)


GILD 82.0199966430664 4511.099815368652
LLY 920.260009765625 6147.336865234375
NVR 9555.009765625 4968.605078125001
MCK 473.80999755859375 3108.193583984375
STAN.L 7.732000122070312 10.335240762801694 2027.7742376616925
JKHY 170.8000030517578 5124.000091552734
601225.SS 25.81999969482422 3.6789010297953917 2207.3406178772348
600276.SS 44.13999938964844 6.289182460459015 1886.7547381377044
600900.SS 29.040000915527344 4.137695218284935 2482.617130970961
MRK 113.87999725341797 1328.9795679473877
VRTX 462.0799865722656 1418.5855587768554
WMT 81.04000091552734 3030.8960342407227
CTRA 22.989999771118164 1494.1200851249694
39736.30340500266


In [29]:
# compute the weight
for stock in cur_investment:
    stock['weight'] = stock['value'] / tot_value

cur_investment

[{'stock': 'GILD',
  'quantity': 55,
  'value': 4511.099815368652,
  'weight': 0.11352590525068118},
 {'stock': 'LLY',
  'quantity': 6.68,
  'value': 6147.336865234375,
  'weight': 0.15470328990039992},
 {'stock': 'NVR',
  'quantity': 0.52,
  'value': 4968.605078125001,
  'weight': 0.125039438809486},
 {'stock': 'MCK',
  'quantity': 6.56,
  'value': 3108.193583984375,
  'weight': 0.07822050159786792},
 {'stock': 'STAN.L',
  'quantity': 196.2,
  'value': 2027.7742376616925,
  'weight': 0.05103077196170197},
 {'stock': 'JKHY',
  'quantity': 30,
  'value': 5124.000091552734,
  'weight': 0.12895009481198094},
 {'stock': '601225.SS',
  'quantity': 600,
  'value': 2207.3406178772348,
  'weight': 0.05554972226227109},
 {'stock': '600276.SS',
  'quantity': 300,
  'value': 1886.7547381377044,
  'weight': 0.047481888763215165},
 {'stock': '600900.SS',
  'quantity': 600,
  'value': 2482.617130970961,
  'weight': 0.062477304586374995},
 {'stock': 'MRK',
  'quantity': 11.67,
  'value': 1328.9795679

In [30]:
import pandas as pd
import numpy as np

data_dir = "./processed_data_128_0928"
S = pd.read_pickle(f'{data_dir}/S.pkl')
mu = np.load(f'{data_dir}/mu.npy')

# load valid_tickers.txt
with open(f'{data_dir}/valid_tickers.txt', 'r') as f:
    valid_tickers = f.read().splitlines()


In [31]:
current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]

In [32]:
def portfolio_volatility(weights, covariance_log_returns):
    covariance_returns = np.exp(covariance_log_returns) - 1
    return np.sqrt(np.dot(weights.T, np.dot(covariance_returns, weights)))

def portfolio_return(weights, log_returns, allow_short=False):
    returns = np.exp(log_returns) - 1
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)


In [33]:
print(portfolio_volatility(current_weight, S))
print(portfolio_return(current_weight, mu))

0.03662256862736397
0.1340878279114054


In [34]:
from scipy.optimize import minimize
import json

def portfolio_volatility_log_return(weights, covariance):
    return np.sqrt(np.dot(weights.T, np.dot(covariance, weights)))

def portfolio_log_return(weights, returns, allow_short=False):
    return np.sum(np.abs(returns)*weights) if allow_short else np.sum(returns*weights)


def min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short=False):
    portfolio_ret = portfolio_log_return(weights, returns, allow_short)
    portfolio_vol = portfolio_volatility_log_return(weights, covariance)
    sharpe_ratio = (portfolio_ret - risk_free_rate) / portfolio_vol
    return -sharpe_ratio # Negate Sharpe ratio because we minimize the function


def optimize_portfolio(returns, covariance, risk_free_rate, allow_short=False):
    num_assets = len(returns)
    args = (returns, covariance, risk_free_rate)

    # Define constraints
    def constraint_sum(weights):
        return np.sum(weights) - 1
    
    constraints = [{'type': 'eq', 'fun': constraint_sum}]

    bounds = tuple((0.0, 0.20) for _ in range(num_assets))

    # Perform optimization
    def objective(weights):
        return min_func_sharpe(weights, returns, covariance, risk_free_rate, allow_short)
    
    iteration = [0]  # mutable container to store iteration count
    def callback(weights):
        iteration[0] += 1
        
        print(f"Iteration: {iteration[0]}, value: {objective(weights)}")

    # Initial guess (equal weights)
    initial_guess = num_assets * [1. / num_assets]

    # Perform optimization
    result = minimize(objective, initial_guess, 
                      method='SLSQP', bounds=bounds, constraints=constraints, callback=callback, options={'maxiter': 300})

    return result

INTEREST_RATE = 0.0497    # Current interest rate accessible for USD
ANNUAL_TRADING_DAYS = 252

def do_optimization(mu, S, final_tickers, period, allow_short):
  riskfree_log_return = np.log(1 + INTEREST_RATE) * period / ANNUAL_TRADING_DAYS
  raw_weights = optimize_portfolio(mu, S, riskfree_log_return, allow_short)
  weights = raw_weights.x
  
  tickers_to_buy = []
  print(f'Starting optimization: allow_short: {allow_short}')
  for index, ticker_name in enumerate(final_tickers):
    weight = weights[index]
    if weight > 1e-3:
      print(f'index: {index} {ticker_name}: weight {weight} exp profit: {mu[index]}, variance: {S[ticker_name][ticker_name]}')
      ticker_info = {'id': ticker_name, 'weight': weight}
      tickers_to_buy.append(ticker_info)

  print(f'expected return in {period} trading days: {portfolio_return(weights, mu)}')
  print(f'volatility of the return in {period} trading days: {portfolio_volatility(weights, S)}')
  print(f'optimization finished for allow_short={allow_short}')
  # print tickers_to_buy in JSON format

  tickers_to_buy_json = json.dumps(tickers_to_buy, indent=4)
  print(tickers_to_buy_json)
  return tickers_to_buy

In [35]:
tickers_to_buy = do_optimization(mu, S, valid_tickers, period=128, allow_short=False)

Iteration: 1, value: -1.4777107370679465
Iteration: 2, value: -1.6065851562677227
Iteration: 3, value: -0.6008091425711545
Iteration: 4, value: -0.5992051012831279
Iteration: 5, value: -1.6418769764807706
Iteration: 6, value: -0.847468053021559
Iteration: 7, value: -1.4039866109351626
Iteration: 8, value: -1.5891852600675322
Iteration: 9, value: -1.9058109054110677
Iteration: 10, value: -1.1213742959981174
Iteration: 11, value: -1.5873609197653546
Iteration: 12, value: -1.5453093035303962
Iteration: 13, value: -1.9068228665219618
Iteration: 14, value: -2.4483604736047178
Iteration: 15, value: -2.5705455671405484
Iteration: 16, value: -2.8606245619357913
Iteration: 17, value: -2.479341337328605
Iteration: 18, value: -2.837699551725213
Iteration: 19, value: -2.5938277621805397
Iteration: 20, value: -2.956421898676583
Iteration: 21, value: -2.983733609665094
Iteration: 22, value: -2.8546399636193187
Iteration: 23, value: -3.023722547701789
Iteration: 24, value: -2.990878094742276
Iteratio

In [36]:
# to find the steps to optimize my current portfolio to the optimized portfolio
# we need to find the difference between the optimized portfolio and the current portfolio
# and then prioritize the changes by the improvement of the portfolio

old_portfolio = pd.DataFrame(cur_investment)
optimized_portfolio = pd.DataFrame(tickers_to_buy)

# Merge the two dataframes to compare weights
comparison = pd.merge(old_portfolio, optimized_portfolio, left_on='stock', right_on='id', how='outer')
comparison['stock'] = comparison['stock'].fillna(comparison['id'])
comparison.fillna(0, inplace=True)  # Assuming no stock means 0 weight
comparison['weight_difference'] = comparison['weight_y'] - comparison['weight_x']

#*comparison['new_quantity'] = comparison['quantity'] * comparison['weight_y'] / comparison['weight_x'] if comparison['weight_x'] != 0 else 0
print(comparison[['stock','weight_x', 'weight_y', 'weight_difference', 'quantity']])

        stock  weight_x  weight_y  weight_difference  quantity
0        GILD  0.113526  0.157569           0.044043     55.00
1         LLY  0.154703  0.075681          -0.079022      6.68
2         NVR  0.125039  0.047666          -0.077373      0.52
3         MCK  0.078221  0.019270          -0.058950      6.56
4      STAN.L  0.051031  0.018415          -0.032616    196.20
5        JKHY  0.128950  0.200000           0.071050     30.00
6   601225.SS  0.055550  0.047125          -0.008424    600.00
7   600276.SS  0.047482  0.022475          -0.025006    300.00
8   600900.SS  0.062477  0.080778           0.018300    600.00
9         MRK  0.033445  0.000000          -0.033445     11.67
10       VRTX  0.035700  0.030247          -0.005453      3.07
11        WMT  0.076275  0.062062          -0.014213     37.40
12       CTRA  0.037601  0.033559          -0.004042     64.99
13       BA.L  0.000000  0.028565           0.028565      0.00
14     FRAS.L  0.000000  0.024153           0.024153   

In [37]:
comparison

,stock,quantity,value,weight_x,id,weight_y,weight_difference
0,GILD,55.00,4511.099815,0.113526,GILD,0.157569,0.044043
1,LLY,6.68,6147.336865,0.154703,LLY,0.075681,-0.079022
2,NVR,0.52,4968.605078,0.125039,NVR,0.047666,-0.077373
3,MCK,6.56,3108.193584,0.078221,MCK,0.019270,-0.058950
4,STAN.L,196.20,2027.774238,0.051031,STAN.L,0.018415,-0.032616
5,JKHY,30.00,5124.000092,0.128950,JKHY,0.200000,0.071050
6,601225.SS,600.00,2207.340618,0.055550,601225.SS,0.047125,-0.008424
7,600276.SS,300.00,1886.754738,0.047482,600276.SS,0.022475,-0.025006
8,600900.SS,600.00,2482.617131,0.062477,600900.SS,0.080778,0.018300
9,MRK,11.67,1328.979568,0.033445,0,0.000000,-0.033445


In [38]:
# Initialize the weights array

current_weight = np.zeros(len(valid_tickers))
for stock in cur_investment:
    if stock["stock"] in valid_tickers:
        current_weight[valid_tickers.index(stock["stock"])] = stock["weight"]


original_sharpe = portfolio_log_return(current_weight, mu) / portfolio_volatility_log_return(current_weight, S)
print(f'Original Sharpe ratio: {original_sharpe}')
# Optimize one stock at a time and adjust other weights proportionally

new_weights = current_weight.copy()
for i in range(15):
    results = []
    print(f'Starting iteration {i} optimization...')
    for index, row in comparison.iterrows():
        new_weights_try = new_weights.copy()
        stock_name = comparison.loc[index, 'stock']
        stock_idx = valid_tickers.index(stock_name)
        # Adjust the targeted stock weight
        new_weights_try[stock_idx] = comparison.loc[index, 'weight_y']
        # Adjust the rest to ensure total sum remains 1
        total_adjusted_weights = np.sum(new_weights_try) - new_weights_try[stock_idx]
        scale_factor = (1 - new_weights_try[stock_idx]) / total_adjusted_weights
        for j in range(len(new_weights_try)):
            if j != stock_idx:
                new_weights_try[j] *= scale_factor

        new_sharpe = portfolio_log_return(new_weights_try, mu) / portfolio_volatility_log_return(new_weights_try, S)
        results.append((comparison.loc[index, 'stock'], new_sharpe, new_sharpe - original_sharpe, new_weights_try, comparison.loc[index, 'weight_difference']))

    results.sort(key=lambda x: x[2], reverse=True)


    stock_name = results[0][0]
    stock_idx = valid_tickers.index(stock_name)
    # Adjust the targeted stock weight
    new_weights = results[0][3]
    new_sharpe = results[0][1]
    # remove the row from comparison
    comparison = comparison[comparison['stock'] != stock_name]

    original_sharpe = new_sharpe

    print(f'Iteration {i} optimization result:')
    print(f'Optimized ticker: {stock_name}')
    print(f'Sharpe ratio: {results[0][1]}')
    print(f'Improvement: {results[0][2]}')
    print(f'Weight difference: {results[0][4]}')
    


Original Sharpe ratio: 3.5260998366686374
Starting iteration 0 optimization...
Iteration 0 optimization result:
Optimized ticker: 601225.SS
Sharpe ratio: 3.5365034509317748
Improvement: 0.010403614263137406
Weight difference: -0.00842427802293242
Starting iteration 1 optimization...
Iteration 1 optimization result:
Optimized ticker: REGN
Sharpe ratio: 3.5411053451964754
Improvement: 0.00460189426470059
Weight difference: 0.002907183234579835
Starting iteration 2 optimization...
Iteration 2 optimization result:
Optimized ticker: FICO
Sharpe ratio: 3.5443489798551866
Improvement: 0.0032436346587112475
Weight difference: 0.0034615258419761997
Starting iteration 3 optimization...
Iteration 3 optimization result:
Optimized ticker: AMGN
Sharpe ratio: 3.545762094339721
Improvement: 0.0014131144845341836
Weight difference: 0.0065719195717303695
Starting iteration 4 optimization...
Iteration 4 optimization result:
Optimized ticker: 600150.SS
Sharpe ratio: 3.547651205112589
Improvement: 0.001889

In [25]:
results-

SyntaxError: invalid syntax (3649125080.py, line 1)

In [ ]:
optimized_portfolio

,id,weight
0,BA.L,0.023168
1,FRAS.L,0.037291
2,STAN.L,0.012661
3,AMGN,0.004777
4,ACGL,0.012257
5,CTRA,0.032321
6,LLY,0.066531
7,FICO,0.011387
8,GILD,0.163008
9,JBL,0.010057
